In [ ]:
---

SELECT COUNT(pt.id) AS posts_cnt
  FROM stackoverflow.posts AS p
  JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
 WHERE pt.type = 'Question'
   AND p.score > 300
    OR p.favorites_count >= 100;
    
---

SELECT ROUND(AVG(t.count))
  FROM
            (SELECT COUNT(id),
                    creation_date::date AS dt
               FROM stackoverflow.posts
              WHERE post_type_id = 1
              GROUP BY creation_date::date
             HAVING creation_date::date BETWEEN '2008-11-01' AND '2008-11-18') AS t
        
---

SELECT COUNT(DISTINCT u.id) AS user_id_cnt
  FROM stackoverflow.users AS u
  JOIN stackoverflow.badges AS b ON u.id = b.user_id
 WHERE b.creation_date::date = u.creation_date::date;
                    
---

SELECT COUNT(t.posts_cnt)
  FROM
      (SELECT p.id AS posts_cnt
         FROM stackoverflow.posts AS p
         JOIN stackoverflow.votes AS v ON p.id = v.post_id
         JOIN stackoverflow.users AS u ON p.user_id = u.id
        WHERE u.display_name LIKE 'Joel Coehoorn'
        GROUP BY p.id
       HAVING COUNT(v.id) >= 1) AS t;
        
---

SELECT *,
   ROW_NUMBER() OVER(ORDER BY id DESC)  AS rank
  FROM stackoverflow.vote_types
 ORDER BY id;
    
---

SELECT *
  FROM
  
(SELECT v.user_id AS user_id,
       COUNT(vt.id) AS cnt
  FROM stackoverflow.votes AS v
  JOIN stackoverflow.vote_types AS vt ON vt.id = v.vote_type_id
 WHERE vt.name LIKE 'Close'
 GROUP BY v.user_id
 ORDER BY cnt DESC
 LIMIT 10
  ) AS t
 ORDER BY t.cnt DESC, t.user_id DESC;
    
---

SELECT user_id,
       badges_cnt,
       DENSE_RANK() OVER(ORDER BY badges_cnt DESC) AS dr
  FROM
      (SELECT user_id,
              COUNT(id) AS badges_cnt
              FROM stackoverflow.badges
             WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
             GROUP BY user_id
             ORDER BY badges_cnt DESC, user_id
             LIMIT 10) AS t;
    
---

SELECT title,
       user_id,
       score,
       ROUND(AVG(SCORE) OVER(PARTITION BY user_id), 0) AS avg_score
  FROM stackoverflow.posts
 WHERE title IS NOT NULL
   AND score != 0;

---

SELECT title
  FROM stackoverflow.posts
 WHERE user_id IN 
                 (SELECT user_id
                    FROM stackoverflow.badges
                   GROUP BY user_id
                  HAVING COUNT(id) > 1000)
    AND title IS NOT NULL;
    
---

SELECT id,
       views,
       CASE WHEN views >= 350 THEN 1
            WHEN views < 350 AND views >= 100 THEN 2
            WHEN views < 100 THEN 3 
         END AS group
  FROM stackoverflow.users
 WHERE location LIKE '%United States%'
   AND views != 0
 GROUP BY id, views;
    
---

WITH tab1 AS
            (SELECT t.id,
                    t.views,
                    t.group,
                    MAX(t.views) OVER (PARTITION BY t.group) AS max
               FROM
                   (SELECT id,
                           views,
                           CASE WHEN views >= 350 THEN 1
                           WHEN views < 350 AND views >= 100 THEN 2
                           WHEN views < 100 THEN 3 
                            END AS group
                      FROM stackoverflow.users
                     WHERE location LIKE '%United States%'
                       AND views != 0
                     GROUP BY id, views) AS t )
SELECT tab1.id,
       tab1.views,
       tab1.group
  FROM tab1
 WHERE tab1.views = tab1.max
 ORDER BY tab1.views DESC, tab1.id;

---

WITH tab1 AS 
(SELECT EXTRACT(DAY FROM creation_date::date) AS dt,
       COUNT(id) AS users_cnt
  FROM stackoverflow.users
 WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
 GROUP BY EXTRACT(DAY FROM creation_date::date))
 
 SELECT dt,
        users_cnt,
        SUM(users_cnt) OVER(ORDER BY dt)
   FROM tab1
  ORDER BY dt;
    
---

WITH p AS 
         (SELECT DISTINCT user_id,
                 MIN(creation_date) OVER(PARTITION BY user_id) AS min
            FROM stackoverflow.posts) 
SELECT user_id,
       min - u.creation_date AS delta
  FROM stackoverflow.users AS u
  JOIN p ON p.user_id = u.id;
    
---

SELECT DATE_TRUNC('month', creation_date)::date AS dt,
       SUM(views_count) AS posts_cnt
  FROM stackoverflow.posts
 GROUP BY DATE_TRUNC('month', creation_date)::date
 ORDER BY SUM(views_count) DESC;

---

SELECT display_name AS name,
       COUNT(DISTINCT p.user_id)
  FROM stackoverflow.posts AS p
  JOIN stackoverflow.users AS u ON p.user_id = u.id
  JOIN stackoverflow.post_types AS pt ON p.post_type_id = pt.id
 WHERE p.creation_date::date BETWEEN u.creation_date::date AND (u.creation_date::date + INTERVAL '1 month')
   AND pt.type LIKE '%Answer%'
 GROUP BY display_name
HAVING COUNT(pt.id) > 100
 ORDER BY display_name;
    
---

WITH tab1 AS
            (SELECT u.id
               FROM stackoverflow.posts AS p
               JOIN stackoverflow.users AS u ON p.user_id = u.id
              WHERE EXTRACT(MONTH FROM u.creation_date::date) = 09
                AND EXTRACT(MONTH FROM p.creation_date::date) = 12
             GROUP BY u.id
             HAVING COUNT(p.id) > 0)
SELECT COUNT(p.id),
       DATE_TRUNC('month', p.creation_date)::date
  FROM stackoverflow.posts AS p
 WHERE p.user_id IN (SELECT * FROM tab1)
   AND EXTRACT(YEAR FROM p.creation_date::date) = 2008
 GROUP BY DATE_TRUNC('month', p.creation_date)::date
 ORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;
        
---

SELECT user_id,
       creation_date AS dt,
       views_count,
       SUM(views_count) OVER(PARTITION BY user_id ORDER BY creation_date) AS views_sum
  FROM stackoverflow.posts;

---

WITH tab1 AS
            (SELECT user_id,
                    COUNT(DISTINCT creation_date::date) AS cnt
               FROM stackoverflow.posts
              WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
              GROUP BY user_id)
SELECT ROUND(AVG(cnt), 0)
  FROM tab1;
    
---

WITH tab1 AS (
              SELECT EXTRACT(MONTH FROM creation_date::date) AS month,
                     COUNT(DISTINCT id)    
                FROM stackoverflow.posts
               WHERE creation_date::date BETWEEN '2008-09-01' AND '2008-12-31'
               GROUP BY month
)

SELECT *,
       ROUND(((count::numeric / LAG(count) OVER (ORDER BY month)) - 1) * 100,2) AS user_growth
FROM tab1;

---

WITH t AS (
SELECT user_id,
       COUNT(DISTINCT id) AS cnt
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY cnt DESC
LIMIT 1),

     t1 AS (
SELECT p.user_id,
       p.creation_date,
       extract('week' from p.creation_date) AS week_number
FROM stackoverflow.posts AS p
JOIN t ON t.user_id = p.user_id
WHERE DATE_TRUNC('month', p.creation_date)::date = '2008-10-01'
           )

SELECT DISTINCT week_number::numeric,
       MAX(creation_date) OVER (PARTITION BY week_number)
FROM t1
ORDER BY week_number;